In [262]:
!pip install snowflake-connector-python pandas scikit-learn

In [263]:
import snowflake.connector
import pandas as pd
import numpy as np

In [264]:
conn = snowflake.connector.connect(
    user = 'chaitanya039',
    password = 'ChaitanyaPansare@9194',
    account = 'BZELMLC-YF23900',
    warehouse = 'COMPUTE_WH',
    database = 'VISTORA_ASSIGNMENT',
    schema = 'FE_SCHEMA'
)

cursor = conn.cursor()

In [265]:
query = 'SELECT * FROM titanic_dataset'
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df.head()


,PASSENGERID,SURVIVED,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN,EMBARKED
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.0500,None,S


In [266]:
df.isnull().sum()

PASSENGERID      0
SURVIVED         0
PCLASS           0
NAME             0
SEX              0
AGE            177
SIBSP            0
PARCH            0
TICKET           0
FARE             0
CABIN          687
EMBARKED         2
dtype: int64

In [267]:
# Feature Transformation
# Fill missing Age with median
df['AGE'] = df['AGE'].fillna(df['AGE'].median())

In [268]:
# Fill missing Embarked with mode
df['EMBARKED'] = df['EMBARKED'].fillna(df['EMBARKED'].mode()[0])

In [269]:
# Remove the cabin column because it contains too many missing values and creates new flag column
df['CABIN_FLAG'] = df['CABIN'].notnull().astype(int)
df = df.drop(columns=['CABIN'])

In [270]:
df.isnull().sum()

PASSENGERID    0
SURVIVED       0
PCLASS         0
NAME           0
SEX            0
AGE            0
SIBSP          0
PARCH          0
TICKET         0
FARE           0
EMBARKED       0
CABIN_FLAG     0
dtype: int64

In [271]:
# let's handle categorical values from here
# Binary encoding for sex
df['SEX'] = df['SEX'].map({ 'male': 0, 'female': 1 })

In [272]:
# One hot encoding for embarked column
df = pd.get_dummies(df, columns=['EMBARKED'], drop_first=True, dtype=int)

In [273]:
df.head()

,PASSENGERID,SURVIVED,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN_FLAG,EMBARKED_Q,EMBARKED_S
0,1,0,3,"Braund, Mr. Owen Harris",0,22.00,1,0,A/5 21171,7.2500,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.00,1,0,PC 17599,71.2833,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.00,0,0,STON/O2. 3101282,7.9250,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.00,1,0,113803,53.1000,1,0,1
4,5,0,3,"Allen, Mr. William Henry",0,35.00,0,0,373450,8.0500,0,0,1


In [274]:
# Create New Feature
df['FAMILY_SIZE'] = df['SIBSP'] + df['PARCH'] + 1;

In [275]:
# Is Alone
df["ISALONE"] = (df["FAMILY_SIZE"] == 1).astype(int)

In [276]:
# Feature Extraction
df['TITLE'] = df['NAME'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df["TITLE"] = df["TITLE"].replace(
    ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 
     'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare'
)
df["TITLE"] = df["TITLE"].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})

In [277]:
# Label Encoding for categorical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['TITLE'] = le.fit_transform(df['TITLE'])

In [278]:
df.head()

,PASSENGERID,SURVIVED,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN_FLAG,EMBARKED_Q,EMBARKED_S,FAMILY_SIZE,ISALONE,TITLE
0,1,0,3,"Braund, Mr. Owen Harris",0,22.00,1,0,A/5 21171,7.2500,0,0,1,2,0,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.00,1,0,PC 17599,71.2833,1,0,0,2,0,3
2,3,1,3,"Heikkinen, Miss. Laina",1,26.00,0,0,STON/O2. 3101282,7.9250,0,0,1,1,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.00,1,0,113803,53.1000,1,0,1,2,0,3
4,5,0,3,"Allen, Mr. William Henry",0,35.00,0,0,373450,8.0500,0,0,1,1,1,2


In [279]:
features = df[[  
    "PASSENGERID",
    "PCLASS",
    "SEX",
    "AGE",
    "SIBSP",
    "PARCH",
    "FARE",
    "CABIN_FLAG",
    "EMBARKED_Q",
    "EMBARKED_S",
    "FAMILY_SIZE",
    "ISALONE",
    "TITLE"
]]

In [280]:
features.head()

,PASSENGERID,PCLASS,SEX,AGE,SIBSP,PARCH,FARE,CABIN_FLAG,EMBARKED_Q,EMBARKED_S,FAMILY_SIZE,ISALONE,TITLE
0,1,3,0,22.00,1,0,7.2500,0,0,1,2,0,2
1,2,1,1,38.00,1,0,71.2833,1,0,0,2,0,3
2,3,3,1,26.00,0,0,7.9250,0,0,1,1,1,1
3,4,1,1,35.00,1,0,53.1000,1,0,1,2,0,3
4,5,3,0,35.00,0,0,8.0500,0,0,1,1,1,2


In [281]:
# Define the SQL insert query
insert_sql = """
INSERT INTO ML_FEATURE_STORE.TITANIC_SCHEMA.TITANIC_FEATURES (
    PASSENGERID, PCLASS, SEX, AGE, SIBSP, PARCH, FARE, CABIN_FLAG, 
    EMBARKED_Q, EMBARKED_S, FAMILY_SIZE, ISALONE, TITLE
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from the features DataFrame into Snowflake
for _, row in features.iterrows():
    cursor.execute(insert_sql, tuple(row))


In [305]:
cursor.execute("SELECT * FROM ML_FEATURE_STORE.TITANIC_SCHEMA.TITANIC_FEATURES")
feature_df = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])

feature_df.head()

,PASSENGERID,PCLASS,SEX,AGE,SIBSP,PARCH,FARE,CABIN_FLAG,EMBARKED_Q,EMBARKED_S,FAMILY_SIZE,ISALONE,TITLE
0,1,3,0,22.0,1,0,7.2500,0,0,1,2,0,2
1,2,1,1,38.0,1,0,71.2833,1,0,0,2,0,3
2,3,3,1,26.0,0,0,7.9250,0,0,1,1,1,1
3,4,1,1,35.0,1,0,53.1000,1,0,1,2,0,3
4,5,3,0,35.0,0,0,8.0500,0,0,1,1,1,2


In [315]:
cursor.execute("SELECT SURVIVED FROM titanic_dataset")
survived = [row[0] for row in cursor.fetchall()]

# Add the 'Survived' column to the feature dataframe
feature_df["SURVIVED"] = survived

# Check the combined data (features + target)
feature_df.head()

,PASSENGERID,PCLASS,SEX,AGE,SIBSP,PARCH,FARE,CABIN_FLAG,EMBARKED_Q,EMBARKED_S,FAMILY_SIZE,ISALONE,TITLE,SURVIVED
0,1,3,0,22.0,1,0,7.2500,0,0,1,2,0,2,0
1,2,1,1,38.0,1,0,71.2833,1,0,0,2,0,3,1
2,3,3,1,26.0,0,0,7.9250,0,0,1,1,1,1,1
3,4,1,1,35.0,1,0,53.1000,1,0,1,2,0,3,1
4,5,3,0,35.0,0,0,8.0500,0,0,1,1,1,2,0


In [317]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the data into features (X) and target (y)
X = feature_df.drop(columns=["PASSENGERID", "SURVIVED"])
y = feature_df["SURVIVED"]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = clf.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))

Model Accuracy: 0.8324022346368715
